In [1]:
from data_generators import get_train_test_addition

#dimensjoner og størrelser til x og y
n_digits = 2
n_max = 3*n_digits
m = 10

#definerer størrelsen på parametermatrisene
d = 15
k = 5
p = 20

#henter treningsdata
data = get_train_test_addition(n_digits,samples_per_batch=100,n_batches_train=4)

In [2]:
from utils import onehot

x = data['x_train'][0]
X = onehot(x,m)
y = data['y_train'][0]

In [3]:
data['x_train'].shape

(4, 100, 6)

In [4]:
from layers import LinearLayer,EmbedPosition,Attention,Softmax,CrossEntropy,FeedForward

embed = EmbedPosition(n_max,m,d)
att1 = Attention(d,k)
ff1 = FeedForward(d,p)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

In [5]:
#vi ser at dLdW_2 tilhørende ff1 (først finner vi det lineære laget l2
#så nøkkel 'w' i params dict-en, så nøkkel 'd' for å finne den deriverte)
#nå er denne bare en nullmatrise 
ff1.l2.params['w']['d']

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [6]:
#"manuelt" forward pass (tilsvarende algoritme 1)
z0 = embed.forward(X)
z1 = att1.forward(z0)
z2 = ff1.forward(z1)
z = un_embed.forward(z2)
Z = softmax.forward(z)

In [7]:
#evaluerer objektfunksjonen
L = loss.forward(Z,y)
print(L)

#finner den deriverte av objektfunksjonen mhp Z
dLdZ = loss.backward()

1.8978686861208496


In [8]:
#"manuelt" backward pass (tilsvarende algoritme 2)
dLdz = softmax.backward(dLdZ)
print('1')
dLdz2 = un_embed.backward(dLdz)
print('2')
dLdz1 = ff1.backward(dLdz2)
print('3')
dLdz0 = att1.backward(dLdz1)
print('4')
embed.backward(dLdz0)

ValueError: operands could not be broadcast together with shapes (100,10,7) (100,10,6) 

In [ ]:
#etter backward pass har dLdW_2 fått verdier
ff1.l2.params['w']['d']

array([[-8.55180647e-06,  2.77222691e-04,  6.20648883e-04,
         3.69191912e-04,  1.53258454e-04,  1.37358689e-04,
         9.81788509e-04,  4.58466752e-04,  5.30768210e-04,
         3.34987189e-04,  8.77523082e-04,  6.68256972e-05,
         6.80254526e-04,  6.23545057e-04,  5.99204086e-04,
         3.41878140e-04,  8.55225161e-04,  1.00841773e-03,
         1.72115406e-04,  6.74327446e-04],
       [ 1.69301213e-03,  1.74353729e-03,  4.67171969e-03,
         4.21596603e-03,  5.17138184e-03,  4.73500389e-03,
         2.29462565e-03,  3.06875754e-03,  3.71466242e-03,
         1.67709129e-03,  8.95939326e-03,  2.21606677e-03,
         4.29504412e-03,  3.47132472e-03,  1.17943195e-03,
         3.45494368e-03,  3.62757126e-03,  4.25837911e-03,
         3.43026511e-03,  3.12284355e-03],
       [-1.49681467e-03, -1.64031980e-03, -4.54380257e-03,
        -4.31646456e-03, -5.38736695e-03, -5.21831538e-03,
        -2.33587444e-03, -3.44090166e-03, -3.90498562e-03,
        -1.50262149e-03, -8.2

In [ ]:
from neural_network import NeuralNetwork

#vi kan samle lagene i en liste som vi bruker for å
#initialisere et nevralt nettverk der vi kan bruke forward() og backward() 
#for å oppnå det samme som vi gjorde manuelt over

layers = [embed,att1,ff1,un_embed,softmax]
nn = NeuralNetwork(layers)

In [ ]:
#forward pass tilsvarende algoritme 1
Z = nn.forward(X)

#beregner loss med CrossEntropy
L = loss.forward(Z,y)
print(L)

#backward pass tilsvarende algoritme 2
dLdZ = loss.backward()
nn.backward(dLdZ)

1.903142451933809
hei


In [ ]:
#før optimering er W_2[0,0] gitt ved
#ff1 er det tredje laget i layers-listen (derav layers[2])
W_2_pre_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_pre_opt[0,0])

0.06310878976328958


In [ ]:
import numpy as np

xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 100
step_size = 0.1

#treningsløkke tilsvarende algoritme 4 (med gradient descent)
for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_gd(step_size)
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

hei


c:\Users\tryme\Documents\GitHub\TMA4320-Introduksjon-til-vitenskapelige-beregninger-prosjekter\Prosjekt2-IndustriellMatematikk\layers.py:137: RuntimeWarning: overflow encountered in exp
  self.P = np.exp(x)
c:\Users\tryme\Documents\GitHub\TMA4320-Introduksjon-til-vitenskapelige-beregninger-prosjekter\Prosjekt2-IndustriellMatematikk\layers.py:139: RuntimeWarning: invalid value encountered in divide
  self.Z = self.P/(Q+self.epsilon)
c:\Users\tryme\Documents\GitHub\TMA4320-Introduksjon-til-vitenskapelige-beregninger-prosjekter\Prosjekt2-IndustriellMatematikk\layers.py:148: RuntimeWarning: overflow encountered in exp
  P = np.exp(self.x)
c:\Users\tryme\Documents\GitHub\TMA4320-Introduksjon-til-vitenskapelige-beregninger-prosjekter\Prosjekt2-IndustriellMatematikk\layers.py:150: RuntimeWarning: invalid value encountered in divide
  S = P/(Q*Q+self.epsilon)


hei
hei
hei
Iterasjon  0  L =  nan 
hei
hei
hei
hei
Iterasjon  1  L =  nan 
hei


KeyboardInterrupt: 

In [ ]:
#etter trening ser vi at W_2 er gitt ved

W_2_post_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_post_opt[0,0])

#dersom differansen er større enn null har dette parameteret endret seg etter 
#gradient descent
print(W_2_post_opt[0,0] - W_2_pre_opt[0,0])




-0.13857317601666091
-0.00010665015797989286
